# for_spss
작성자: 황승현
- 목적
    - preprocessing.ipynb + merge_how_left.ipynb 를 합치기
    - 식영과 논문 작성을 위한 데이터 정리
    - spss...?
- 해야하는 것
    - 여성만 + 유전체 데이터 붙이기
    - 결측값 제거
    - 표준화 등 하지 않기 (원본 수치 유지)

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd

In [2]:
# 데이터셋 불러오기
dataset = pd.read_csv("dataset/Dataset_OP_230614_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' '])
dataset.head()

C:\Users\sh22h\AppData\Local\Temp\ipykernel_27628\2482683002.py:2: DtypeWarning: Columns (7,9,156,299) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("dataset/Dataset_OP_230614_raw.csv", index_col=0, na_values=[77777, 99999, '#NULL!', ' '])


,AS1_SEX,AS1_AGE,AS1_FAMNUM,AS1_HAND,AS1_SEAS,AS1_MARRYA,AS1_MARRYAETC,AS1_RELIG,AS1_RELIGETC,AS1_RELIGPRT,...,AS1_INSM.1,AS1_SLPHRD.1,AS1_SLPAMSF.1,AS1_ICOFF_1,AS1_ICOFF_2,AS1_W098.1,AS1_MNSAG.2,AS1_BRSTF,AS1_BRSTC,AS1_EGFR
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH2307795046,1,53,4.0,1.0,3.0,2.0,NaN,3.0,NaN,1.0,...,1.0,1.0,2.0,2,2,0.225000,NaN,NaN,NaN,81.027088
NIH2307713334,2,44,2.0,1.0,2.0,2.0,NaN,4.0,NaN,NaN,...,1.0,1.0,2.0,3,2,0.578571,13.0,NaN,NaN,223.651123
NIH2307455306,2,43,6.0,1.0,2.0,2.0,NaN,3.0,NaN,1.0,...,1.0,1.0,1.0,1,1,0.000000,20.0,2.0,3.0,225.227717
NIH2307778268,1,61,4.0,1.0,4.0,2.0,NaN,3.0,NaN,1.0,...,1.0,1.0,2.0,2,2,0.225000,NaN,NaN,NaN,134.847877
NIH2307137554,2,64,4.0,1.0,1.0,2.0,NaN,3.0,NaN,1.0,...,1.0,1.0,2.0,4,2,0.000000,17.0,2.0,2.0,194.337250


In [3]:
# 특성별로 분류
var_to_use = pd.read_csv("dataset/var_to_use_2023-06-13T1650I.csv")
var_to_use

,binary,cath0,cath1,cnt,dependent
0,AS1_SEX,AS1_EDUA,AS1_INCOME,AS1_AGE,AS1_DT
1,AS1_HT,AS1_DRINK,AS1_PHYSTB,AS1_TOTALC,AS1_MT
2,AS1_DM,AS1_SMOKEA,AS1_PHYSIT,AS1_PMAG_C,NaN
3,AS1_AL,AS1_FMOSREL_N,AS1_PHYACTL,AS1_MNSAG,NaN
4,AS1_MI,AS1_FMOSREL_O,AS1_PHYACTM,AS1_CHILD_P,NaN
...,...,...,...,...,...
70,NaN,NaN,NaN,AS1_BPSIT1RS,NaN
71,NaN,NaN,NaN,AS1_BPSIT1SYS,NaN
72,NaN,NaN,NaN,AS1_BPSIT1LD,NaN
73,NaN,NaN,NaN,AS1_BPSIT1RD,NaN


In [4]:
# 유전체 SNP 매칭
gen_dict = {3335: "SNP_A-2181021",
            214246: "SNP_A-1809518",
            262783: "SNP_A-2130710",
            35799: "SNP_A-2263153",
            154529: "SNP_A-2310995",
            118171: "SNP_A-1922415",
            182907: "SNP_A-2266073",
            122732: "SNP_A-1984271",
            283204: "SNP_A-2218697",
            217552: "SNP_A-1850320",
            335062: "SNP_A-4262878",
            335032: "SNP_A-4299800",
            335031: "SNP_A-2242511"
            }

## 데이터 가공

1. 남성 제거 (헉!)
    - 폐경 전 / 후 여성
    - 폐경 여부(AS1_PMYN_C) 알 수 없으면 DROP
2. 종속변수(골밀도 수치) 결측값 DROP
    - 골다공증 여부 (0 / 1)
3. 기존 데이터로 새로운 데이터 만들기
    - 질환유무, 종양 유무
    - 약물력
    - 가족력
    - 여성력
4. 타입별 분리

### 남성 제거, 종속변수 설정

In [5]:
# 남성(1) drop, 여성(2)만 가져옴
dataset = dataset[dataset['AS1_SEX'] == 2]

# 폐경여부 결측이면 drop
dataset.dropna(subset=['AS1_PMYN_C'], inplace=True)

# 종속변수의 결측값 둘 중 하나라도 있으면 drop
dataset.dropna(subset=['AS1_DT', 'AS1_MT'], how="any", inplace=True)

# # 종속변수의 결측값 둘 다 있으면 drop
# dataset.dropna(subset=['AS1_DT', 'AS1_MT'], how="all", inplace=True)

# 새로운 종속변수 만들기 OP: 골다공증 여부
dataset['OP'] = np.where((dataset['AS1_DT'] <= -2.5) | (dataset['AS1_MT'] <= -2.5), 1, 0)


dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4652 entries, NIH2307713334 to NIH2307534318
Columns: 839 entries, AS1_SEX to OP
dtypes: float64(795), int32(1), int64(38), object(5)
memory usage: 29.8+ MB


### 기존 데이터로 새로운 데이터 만들기

In [6]:
# 질환유무 가공
disease_list = np.array(['HT', 'DM', 'AL', 'MI', 'TH', # 질환 목록
                         'CH', 'CD', 'LP', 'AS', 'CL',
                         'KD', 'CV', 'GT' ]) # UL, PV, HN, TOTCA1 제외

dataset['AS1_ARRM'] = np.where((dataset['AS1_JOAR'] == 2) |
                               (dataset['AS1_JORM'] == 2) |
                               (dataset['AS1_TRTAR'] == 2),
                               1, 0) # 관절혐은 조건 3가지 만족하면 1 아니면 0

for x in disease_list:
    dataset['AS1_'+x] = np.where((dataset['AS1_PD'+x] == 2) |
                                 (dataset['AS1_TRT'+x] == 2),
                                 1, 0) # 질환여부 정의. AS1_PDㅇㅇ 또는 AS1_TRTㅇㅇ 이 2이면 1 아니면 0

In [7]:
# 약물력 가공
drug_list = np.array(['ST', 'CP', 'INS', 'HT', 'AR',
                      'TH', 'FH', 'OS', 'STK', 'AS',
                      'LP', 'SP', 'SL'])  # UL, DM, CP, HP 제외 SP, SL 추가
for x in drug_list:
    dataset['AS1_DR'+x] = np.where((dataset['AS1_DRUG'+x] == 2)
                                   | (dataset['AS1_DRUG'+x+'CU'] == 2),
                                   1, 0)


In [8]:
# 와 ChatGPT가 아니었다면 np.all과 np.any를 몰랐을거야!
# 가족력 가공
# N: 가족력 없음, P: parent 부모, S: sibling 형제자매, O: other 기타
# N: REL1A~4A까지 99999
# P: REL1A~4A중 1, 2 있을 때
# S: REL1A~4A중 3 있을 때
# O: REL1A~4A중 4 있을 때

# disease_dict = {
#     'HT': 4, 'DM': 4, 'CVA': 3, 'HE': 2, 'OS': 2,
#     'CVB': 2, 'CD': 2, 'PV': 2, 'LP': 2, 'GT': 2,'CH': 1
# }

# OS만 쓰기 때문에 반복문 안 돌리고 하나만 있으면 될 듯
disease_dict = { 'OS': 2 }

for disease, num_family in disease_dict.items():
    dataset[f'AS1_FM{disease}REL_N'] = np.all(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] == 99999, axis=1).astype(int)
    dataset[f'AS1_FM{disease}REL_P'] = np.any(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] <= 2, axis=1).astype(int)
    dataset[f'AS1_FM{disease}REL_S'] = np.any(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] == 3, axis=1).astype(int)
    dataset[f'AS1_FM{disease}REL_O'] = np.any(dataset[[f'AS1_FM{disease}REL{i}A' for i in range(1, num_family+1)]] == 4, axis=1).astype(int)

dataset.head()

,AS1_SEX,AS1_AGE,AS1_FAMNUM,AS1_HAND,AS1_SEAS,AS1_MARRYA,AS1_MARRYAETC,AS1_RELIG,AS1_RELIGETC,AS1_RELIGPRT,...,AS1_DROS,AS1_DRSTK,AS1_DRAS,AS1_DRLP,AS1_DRSP,AS1_DRSL,AS1_FMOSREL_N,AS1_FMOSREL_P,AS1_FMOSREL_S,AS1_FMOSREL_O
DIST_ID,,,,,,,,,,,,,,,,,,,,,
NIH2307713334,2,44,2.0,1.0,2.0,2.0,NaN,4.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
NIH2307455306,2,43,6.0,1.0,2.0,2.0,NaN,3.0,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
NIH2307137554,2,64,4.0,1.0,1.0,2.0,NaN,3.0,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
NIH2307649667,2,65,2.0,1.0,3.0,2.0,NaN,2.0,NaN,2.0,...,0,0,0,0,0,0,0,0,0,0
NIH2307899915,2,57,6.0,1.0,3.0,2.0,NaN,3.0,NaN,2.0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
dataset[[f'AS1_FM{disease}REL_{family}' for disease in disease_dict.keys() for family in ['N', 'P', 'S', 'O']]]

,AS1_FMOSREL_N,AS1_FMOSREL_P,AS1_FMOSREL_S,AS1_FMOSREL_O
DIST_ID,,,,
NIH2307713334,0,0,0,0
NIH2307455306,0,0,0,0
NIH2307137554,0,0,0,0
NIH2307649667,0,0,0,0
NIH2307899915,0,1,0,0
...,...,...,...,...
NIH2307591031,0,0,0,0
NIH2307808901,0,0,0,0
NIH2307930134,0,0,0,0


In [10]:
# 여성 관련 약물력

for x in ['CP', 'FH']:
    dataset['AS1_DR'+x] = np.where((dataset['AS1_DRUG'+x] == 2) | (dataset['AS1_DRUG'+x+'CU'] == 2), 1, 0)

In [11]:
# 앉은 자세 혈압

dataset['AS1_BPSIT1SYS'] = np.where(dataset['AS1_BPSIT1LS'] > dataset['AS1_BPSIT1RS'], dataset['AS1_BPSIT1LS'], dataset['AS1_BPSIT1RS'])
dataset['AS1_BPSIT1DIA'] = np.where(dataset['AS1_BPSIT1LD'] > dataset['AS1_BPSIT1RD'], dataset['AS1_BPSIT1LD'], dataset['AS1_BPSIT1RD'])

### 타입 별 분류

In [12]:
# var_to_use의 각 열 가져옴

raw_binary = dataset.reindex(columns=(var_to_use['binary'].dropna()))
raw_cath0 = dataset.reindex(columns=(var_to_use['cath0'].dropna()))
raw_cath1 = dataset.reindex(columns=(var_to_use['cath1'].dropna()))
raw_cnt = dataset.reindex(columns=(var_to_use['cnt'].dropna()))
raw_label = dataset.reindex(columns=(var_to_use['dependent'].dropna()))

raw_label.head()

dependent,AS1_DT,AS1_MT
DIST_ID,,
NIH2307713334,-0.4,0.4
NIH2307455306,0.0,-1.0
NIH2307137554,-1.8,-1.9
NIH2307649667,-2.0,-1.7
NIH2307899915,0.1,-0.1


In [13]:
print(raw_binary.isnull().sum())
print(raw_cath0.isnull().sum())
print(raw_cath1.isnull().sum())
print(raw_cnt.isnull().sum())
print(raw_label.isnull().sum())

binary
AS1_SEX         0
AS1_HT          0
AS1_DM          0
AS1_AL          0
AS1_MI          0
AS1_TH          0
AS1_CH          0
AS1_CD          0
AS1_LP          0
AS1_AS          0
AS1_CL          0
AS1_KD          0
AS1_CV          0
AS1_GT          0
AS1_ARRM        0
AS1_DRST        0
AS1_DRCP        0
AS1_DRINS       0
AS1_DRHT        0
AS1_DRAR        0
AS1_DRTH        0
AS1_DRFH        0
AS1_DROS        0
AS1_DRSTK       0
AS1_DRAS        0
AS1_DRLP        0
AS1_DRSP        0
AS1_DRSL        0
AS1_PMYN_C      0
AS1_PREG       36
AS1_HYST      179
AS1_OBGYOP    386
AS1_BRCA      230
AS1_BRSTF     615
AS1_INSM       32
AS1_SLPHRD     43
dtype: int64
cath0
AS1_EDUA           32
AS1_DRINK          25
AS1_SMOKEA         65
AS1_FMOSREL_N       0
AS1_FMOSREL_O       0
AS1_FMOSREL_P       0
AS1_FMOSREL_S       0
AS1_HYSTOVARYW    498
AS1_OVARYW        510
dtype: int64
cath1
AS1_INCOME     90
AS1_PHYSTB     49
AS1_PHYSIT     51
AS1_PHYACTL    55
AS1_PHYACTM    97
AS1_PHYACTH    76
A

In [14]:
raw_binary.mode()

binary,AS1_SEX,AS1_HT,AS1_DM,AS1_AL,AS1_MI,AS1_TH,AS1_CH,AS1_CD,AS1_LP,AS1_AS,...,AS1_DRSP,AS1_DRSL,AS1_PMYN_C,AS1_PREG,AS1_HYST,AS1_OBGYOP,AS1_BRCA,AS1_BRSTF,AS1_INSM,AS1_SLPHRD
0,2,0,0,0,0,0,0,0,0,0,...,0,0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0


In [15]:
# 원 데이터의 결측값 모두 0으로 대치
binary = raw_binary.fillna(raw_binary.mode().iloc[0]) # 결측값 xmode().iloc[0]
cath0 = raw_cath0.fillna(raw_cath0.mode().iloc[0])
cath1 = raw_cath1.fillna(raw_cath1.mode().iloc[0])
cnt = raw_cnt.fillna(raw_cnt.mean())
label = raw_label

In [16]:
print(binary.isnull().sum())
print(cath0.isnull().sum())
print(cath1.isnull().sum())
print(cnt.isnull().sum())
print(label.isnull().sum())

binary
AS1_SEX       0
AS1_HT        0
AS1_DM        0
AS1_AL        0
AS1_MI        0
AS1_TH        0
AS1_CH        0
AS1_CD        0
AS1_LP        0
AS1_AS        0
AS1_CL        0
AS1_KD        0
AS1_CV        0
AS1_GT        0
AS1_ARRM      0
AS1_DRST      0
AS1_DRCP      0
AS1_DRINS     0
AS1_DRHT      0
AS1_DRAR      0
AS1_DRTH      0
AS1_DRFH      0
AS1_DROS      0
AS1_DRSTK     0
AS1_DRAS      0
AS1_DRLP      0
AS1_DRSP      0
AS1_DRSL      0
AS1_PMYN_C    0
AS1_PREG      0
AS1_HYST      0
AS1_OBGYOP    0
AS1_BRCA      0
AS1_BRSTF     0
AS1_INSM      0
AS1_SLPHRD    0
dtype: int64
cath0
AS1_EDUA          0
AS1_DRINK         0
AS1_SMOKEA        0
AS1_FMOSREL_N     0
AS1_FMOSREL_O     0
AS1_FMOSREL_P     0
AS1_FMOSREL_S     0
AS1_HYSTOVARYW    0
AS1_OVARYW        0
dtype: int64
cath1
AS1_INCOME     0
AS1_PHYSTB     0
AS1_PHYSIT     0
AS1_PHYACTL    0
AS1_PHYACTM    0
AS1_PHYACTH    0
AS1_ICOFF_1    0
dtype: int64
cnt
AS1_AGE          0
AS1_TOTALC       0
AS1_PMAG_C       0
AS1_MN

## 데이터셋에 유전체 붙이기

In [17]:
# 유전체 데이터프레임 만들기
df_all_gen = pd.DataFrame()

for v in gen_dict.values():
    temp = pd.read_csv(f"./dataset/genome/result_{v}.csv", index_col=1, low_memory=False).dropna()
    temp.rename(columns={"SNP": v}, inplace=True)
    df_all_gen = pd.concat([df_all_gen, temp[v]], axis=1)

df_all_gen.head()

,SNP_A-2181021,SNP_A-1809518,SNP_A-2130710,SNP_A-2263153,SNP_A-2310995,SNP_A-1922415,SNP_A-2266073,SNP_A-1984271,SNP_A-2218697,SNP_A-1850320,SNP_A-4262878,SNP_A-4299800,SNP_A-2242511
NIH2307537078,C C,G A,G G,A A,C C,T T,G G,C C,A A,A G,T G,G G,A A
NIH2307151907,A A,G A,G T,A A,T C,T T,G G,G G,A A,G G,T G,G G,A A
NIH2307876488,C C,G G,T T,A A,C C,T T,G G,C C,G A,G G,T G,G G,A A
NIH2307009808,C C,G A,G T,A A,C C,T T,G G,G C,A A,G G,T G,G G,A A
NIH2307798369,C C,A A,T T,A A,C C,T T,G G,G C,G A,G G,G G,G G,A A


In [18]:
diet_and_gen = pd.concat([binary, cath0, cath1, cnt, df_all_gen], axis=1)
diet_and_gen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5217 entries, NIH2307713334 to NIH2307027315
Columns: 140 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(127), object(13)
memory usage: 5.6+ MB


In [19]:
diet_and_gen.dropna().info()
# diet_and_gen.dropna(subset=['AS1_PMYN_C'],).info()

<class 'pandas.core.frame.DataFrame'>
Index: 4093 entries, NIH2307713334 to NIH2307534318
Columns: 140 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(127), object(13)
memory usage: 4.4+ MB


In [23]:
diet_and_gen.dropna(inplace=True)

## 폐경여부로 분리

In [24]:
meno1 = dataset['AS1_PMYN_C'] == 1
meno2 = dataset['AS1_PMYN_C'] == 2

X_premeno_woman = diet_and_gen.loc[meno1]
X_postmeno_woman = diet_and_gen.loc[meno2]
y_premeno_woman = dataset['OP'].loc[meno1]
y_postmeno_woman = dataset['OP'].loc[meno2]
print(f"폐경 전 여성의 골다공증:\n {y_premeno_woman.value_counts()}"
      f"\n\n폐경 후 여성의 골다공증:\n {y_postmeno_woman.value_counts()}")

폐경 전 여성의 골다공증:
 OP
0    1651
1      67
Name: count, dtype: int64

폐경 후 여성의 골다공증:
 OP
0    2109
1     825
Name: count, dtype: int64


In [26]:
X_premeno_woman.to_csv('./dataset/scaled/X_premeno_2023824T1810I.csv')
X_postmeno_woman.to_csv('./dataset/scaled/X_postmeno_20230824T1810I.csv')
y_premeno_woman.to_csv('./dataset/scaled/y_premeno_20230824T1810I.csv')
y_postmeno_woman.to_csv('./dataset/scaled/y_postmeno_20230824T1810I.csv')

In [27]:
X = diet_and_gen[diet_and_gen['AS1_PMYN_C'] != 0]
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4093 entries, NIH2307713334 to NIH2307534318
Columns: 140 entries, AS1_SEX to SNP_A-2242511
dtypes: float64(127), object(13)
memory usage: 4.4+ MB


In [28]:
X.to_csv('./dataset/정규화안했음_20230824T1810I.csv')